In [42]:
import scipy.io
from sklearn.model_selection import KFold, train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from skfeature.function.similarity_based import fisher_score
import numpy as np
from sklearn.metrics import classification_report

In [43]:
    # load data
    mat = scipy.io.loadmat('COIL20.mat')
    X = mat['X']    # data
    X = X.astype(float)
    y = mat['Y']    # label
    y = y[:, 0]

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    n_samples, n_features = X_train.shape    # number of samples and number of features

    # split data into 10 folds
    kf = KFold(n_splits=10, shuffle=True)
    ss = kf.split(X_train)

    # perform evaluation on classification task
    num_fea = 100    # number of selected features


In [44]:
cv_models = []

def feature_ranking(score):
    """
    Rank features in descending order according to fisher score, the larger the fisher score, the      more important the
    feature is
    """
    idx = np.argsort(score, 0)
    return idx[::-1]

for train, test in ss:
    clf = svm.LinearSVC()    # linear SVM
    # obtain the score of each feature on the training set
    score = fisher_score.fisher_score(X_train[train], y_train[train])



    # rank features in descending order according to score
    idx = feature_ranking(score)

    # obtain the dataset on the selected features
    selected_features = X_train[:, idx[0:num_fea]]

    # train a classification model with the selected features on the training dataset
    clf.fit(selected_features[train], y_train[train])

    # predict the class labels of test data
    y_predict = clf.predict(selected_features[test])

    # obtain the classification accuracy on the test data
    acc = accuracy_score(y_train[test], y_predict)
    cv_models.append({
        "model": clf,
        "score": acc,
        "best_features": idx[0:num_fea]
    })


In [45]:
cv_models = sorted(cv_models, key=lambda x: x["score"], reverse=True)
cv_models

[{'model': LinearSVC(),
  'score': 0.9814814814814815,
  'best_features': array([   3,  881,  966, 1013,  991,  150,   23,  159,  392,  130,   84,
          254,  504,  714,  777,  828,  858,  867,  880,  897,  892,  887,
          857,  886,  926,  952,  977,  983,  925,  438,    4,    2,  202,
         1011, 1020,  980,  148,   37,  140,  216,  117,  147,  210,  232,
          227,  437,  570,  644,  684,  656,  662,  668,  710,  726,  764,
          797,  869,  924,  939,  972,  971,  986,  955,    5,  938, 1021,
         1018,  590,   76,  141,  261,  322,  215,  352,  433,  354,  223,
          239,  545,  508,  571,  580,  490,  428,  391,  394,  393,  535,
          741,  868,  920,  936,  950,  953,  981,  303,  945, 1023,  914,
          287])},
 {'model': LinearSVC(),
  'score': 0.9722222222222222,
  'best_features': array([   4,  784,  914, 1012, 1000,  148,   18,  139,  357,   95,   62,
          230,  516,  702,  739,  827,  858,  870,  878,  899,  903,  894,
          865

In [46]:
best_model = cv_models[0]["model"]
best_fetures = cv_models[0]["best_features"]
X_test_ = X_test[:, best_fetures]

In [47]:
y_predict_ = best_model.predict(X_test_)

In [48]:
print(classification_report(y_test, y_predict_))

precision    recall  f1-score   support

           1       0.00      0.00      0.00        23
           2       0.00      0.00      0.00        12
           3       0.04      0.04      0.04        24
           4       0.00      0.00      0.00        15
           5       0.08      0.20      0.12        15
           6       0.00      0.00      0.00        25
           7       0.50      0.21      0.30        14
           8       0.49      1.00      0.66        19
           9       0.04      0.22      0.07        18
          10       0.00      0.00      0.00        17
          11       0.00      0.00      0.00        24
          12       0.00      0.00      0.00        16
          13       0.00      0.00      0.00        15
          14       0.00      0.00      0.00        19
          15       0.00      0.00      0.00        14
          16       0.00      0.00      0.00        21
          17       0.00      0.00      0.00        22
          18       0.14      0.40      0.